<a href="https://colab.research.google.com/github/mariam7705/dev-project/blob/main/WEEK_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

# Load datasets for each city
df_city1 = pd.read_csv('/content/delivery_cq.csv')
df_city2 = pd.read_csv('/content/delivery_delivery_jl.csv')
df_city3 = pd.read_csv('/content/delivery_hz.csv')
df_city4 = pd.read_csv('/content/delivery_sh.csv')
df_city5 = pd.read_csv('/content/delivery_yt.csv')

In [5]:
df_city1['city'] = 'City1'
df_city2['city'] = 'City2'
df_city3['city'] = 'City3'
df_city4['city'] = 'City4'
df_city5['city'] = 'City5'

In [6]:
merged_df = pd.concat([df_city1, df_city2, df_city3, df_city4, df_city5], ignore_index=True)

In [7]:
print(merged_df.info())
print(merged_df['city'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4514661 entries, 0 to 4514660
Data columns (total 17 columns):
 #   Column             Dtype  
---  ------             -----  
 0   order_id           int64  
 1   region_id          int64  
 2   city               object 
 3   courier_id         int64  
 4   lng                float64
 5   lat                float64
 6   aoi_id             int64  
 7   aoi_type           int64  
 8   accept_time        object 
 9   accept_gps_time    object 
 10  accept_gps_lng     float64
 11  accept_gps_lat     float64
 12  delivery_time      object 
 13  delivery_gps_time  object 
 14  delivery_gps_lng   float64
 15  delivery_gps_lat   float64
 16  ds                 int64  
dtypes: float64(6), int64(6), object(5)
memory usage: 585.6+ MB
None
city
City3    1861600
City4    1483864
City1     931351
City5     206431
City2      31415
Name: count, dtype: int64


In [10]:
merged_df.to_csv('merged_city_data.csv', index=False)

In [11]:
merged_df = merged_df.drop_duplicates()

In [12]:
# Load datasets for each city
df_pickup1 = pd.read_csv('/content/pickup_cq.csv')
df_pickup2 = pd.read_csv('/content/pickup_jl.csv')
df_pickup3 = pd.read_csv('/content/pickup_hz.csv')
df_pickup4 = pd.read_csv('/content/pickup_sh.csv')
df_pickup5 = pd.read_csv('/content/pickup_yt.csv')
df_pickup4 = pd.read_csv('/content/pickup_sh.csv', low_memory=False)



In [13]:
df_pickup1['city'] = 'City1'
df_pickup2['city'] = 'City2'
df_pickup3['city'] = 'City3'
df_pickup4['city'] = 'City4'
df_pickup5['city'] = 'City5'


In [14]:
pickup_merged_df = pd.concat([df_pickup1, df_pickup2, df_pickup3, df_pickup4, df_pickup5], ignore_index=True)


In [15]:
pickup_merged_df = pickup_merged_df.drop_duplicates()


In [16]:
print(merged_df.info())
pickup_merged_df.to_csv('merged_pickup_data.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4514661 entries, 0 to 4514660
Data columns (total 17 columns):
 #   Column             Dtype  
---  ------             -----  
 0   order_id           int64  
 1   region_id          int64  
 2   city               object 
 3   courier_id         int64  
 4   lng                float64
 5   lat                float64
 6   aoi_id             int64  
 7   aoi_type           int64  
 8   accept_time        object 
 9   accept_gps_time    object 
 10  accept_gps_lng     float64
 11  accept_gps_lat     float64
 12  delivery_time      object 
 13  delivery_gps_time  object 
 14  delivery_gps_lng   float64
 15  delivery_gps_lat   float64
 16  ds                 int64  
dtypes: float64(6), int64(6), object(5)
memory usage: 585.6+ MB
None


In [ ]:
# Load both merged files
pickup_df = pd.read_csv('merged_pickup_data.csv')
delivery_df = pd.read_csv('merged_city_data.csv')

# Merge based on common key
merged_df = pd.merge(pickup_df, delivery_df, on='order_id', how='inner')

# Save final cleaned file
merged_df.to_csv('final_cleaned_dataset.csv', index=False)


In [ ]:
import pandas as pd

# Step 1: Show missing value report
print("Missing Values Before Cleaning:")
print(merged_df.isnull().sum())


# Step 3: Fill numerical columns (non-GPS) with median
merged_df = merged_df.fillna(merged_df.median(numeric_only=True))

# Step 4: Fill categorical columns with mode (no chained assignment)
for col in merged_df.select_dtypes(include='object'):
    mode_val = merged_df[col].mode()
    if not mode_val.empty:
        merged_df[col] = merged_df[col].fillna(mode_val[0])

# Step 5: Leave GPS columns untouched for now (they still contain NaNs)
# They will be handled in future feature engineering/modeling stages

# Step 6: Save cleaned file
merged_df.to_csv("final_cleaned_dataset.csv", index=False)

# Step 7: Show post-cleanup info
print("✅ Data cleaned and saved. GPS columns retained.")
print("Updated missing value summary:")
print(merged_df.isnull().sum())


In [ ]:



merged_df['pickup_time_x'] = pd.to_datetime(merged_df['pickup_time_x'], errors='coerce')
merged_df['pickup_weekday'] = merged_df['pickup_time_x'].dt.weekday
merged_df['is_weekend'] = merged_df['pickup_weekday'].isin([5, 6])
merged_df['pickup_date'] = merged_df['pickup_time_x'].dt.date
packages_per_courier = merged_df.groupby(['courier_id', 'pickup_date']).size().reset_index(name='packages_assigned')
merged_df = pd.merge(merged_df, packages_per_courier, on=['courier_id', 'pickup_date'], how='left')


In [ ]:
print(merged_df.columns.tolist())



LOCATION DATA NORMALIZATION


In [ ]:

merged_df['lat_x'] = pd.to_numeric(merged_df['lat_x'], errors='coerce')
merged_df['lng_x'] = pd.to_numeric(merged_df['lng_x'], errors='coerce')
merged_df.dropna(subset=['lat_x', 'lng_x'], inplace=True)



In [ ]:
merged_df['lat_y'] = pd.to_numeric(merged_df['lat_y'], errors='coerce')
merged_df['lng_y'] = pd.to_numeric(merged_df['lng_y'], errors='coerce')
merged_df.dropna(subset=['lat_y', 'lng_y'], inplace=True)


. Standardizing Column Names and Types

In [ ]:
# Make column names lowercase and replace spaces
merged_df.columns = [col.strip().lower().replace(' ', '_') for col in merged_df.columns]

# Use courier_id_x from pickup dataset as the main courier ID
merged_df['courier_id'] = merged_df['courier_id_x'].astype(str)


In [11]:
# See first few rows
merged_df.head()


,order_id,region_id,city,courier_id,lng,lat,aoi_id,aoi_type,accept_time,accept_gps_time,accept_gps_lng,accept_gps_lat,delivery_time,delivery_gps_time,delivery_gps_lng,delivery_gps_lat,ds
0,2031782,10,City1,73,108.71571,30.90228,50,14,10-22 10:26:00,10-22 10:26:00,108.71826,30.95587,10-22 17:04:00,10-22 17:04:00,108.66361,30.96702,1022
1,4285071,10,City1,3605,108.71639,30.90269,50,14,09-07 10:13:00,09-07 10:13:00,108.71791,30.95635,09-09 15:44:00,09-09 15:44:00,108.71644,30.90266,907
2,4056800,10,City1,3605,108.71645,30.90259,50,14,06-26 09:49:00,06-26 09:49:00,108.71798,30.95635,06-27 16:03:00,06-27 16:03:00,108.71647,30.90251,626
3,3589481,10,City1,3605,108.71650,30.90347,50,14,09-11 11:01:00,09-11 11:01:00,108.71823,30.95596,09-13 17:14:00,09-13 17:14:00,108.71650,30.90341,911
4,2752329,10,City1,3605,108.71608,30.90409,50,14,10-01 09:52:00,10-01 09:52:00,108.71820,30.95598,10-01 18:30:00,10-01 18:30:00,108.71413,30.90397,1001


In [16]:
print("Dataset shape:", merged_df.shape)
merged_df.info()


Dataset shape: (4514661, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4514661 entries, 0 to 4514660
Data columns (total 17 columns):
 #   Column             Dtype  
---  ------             -----  
 0   order_id           int64  
 1   region_id          int64  
 2   city               object 
 3   courier_id         int64  
 4   lng                float64
 5   lat                float64
 6   aoi_id             int64  
 7   aoi_type           int64  
 8   accept_time        object 
 9   accept_gps_time    object 
 10  accept_gps_lng     float64
 11  accept_gps_lat     float64
 12  delivery_time      object 
 13  delivery_gps_time  object 
 14  delivery_gps_lng   float64
 15  delivery_gps_lat   float64
 16  ds                 int64  
dtypes: float64(6), int64(6), object(5)
memory usage: 585.6+ MB


In [1]:
# Any remaining NaNs?
missing_summary = merged_df.isnull().sum()
print("Missing values per column:\n", missing_summary[missing_summary > 0])


NameError: name 'merged_df' is not defined

In [ ]:
expected_columns = [
    'pickup_hour', 'pickup_day', 'pickup_weekday', 'is_weekend',
    'packages_assigned', 'has_gps_data', 'courier_id', 'pickup_time_x', 'delivery_time'
]

for col in expected_columns:
    print(f"{col}: {col in merged_df.columns}")


In [ ]:
print(merged_df['city_x'].value_counts())
print(merged_df['city_y'].value_counts())


In [2]:
merged_df.to_csv("final_cleaned_data.csv", index=False)


NameError: name 'merged_df' is not defined